In [2]:
import torch
import torchvision
import numpy as np
import time
import cv2
from utils import preprocess
import traitlets
import smbus

class Racecar(traitlets.HasTraits):
    steering = traitlets.Float()
    throttle = traitlets.Float()

    @traitlets.validate('steering')
    def _clip_steering(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))

    @traitlets.validate('throttle')
    def _clip_throttle(self, proposal):
        return max(-1.0, min(1.0, proposal['value']))

class NvidiaRacecar(Racecar):
    i2c_address = traitlets.Integer(default_value=0x40)
    steering_gain = traitlets.Float(default_value=-0.65)
    steering_offset = traitlets.Float(default_value=0.17)
    steering_channel = traitlets.Integer(default_value=0)
    throttle_gain = traitlets.Float(default_value=0.8)
    throttle_channel = traitlets.Integer(default_value=1)

    def __init__(self, *args, **kwargs):
        super(NvidiaRacecar, self).__init__(*args, **kwargs)
        self.bus = smbus.SMBus(7)
        self.center_pulse = 1500
        self._initialize_pca9685()

    def _initialize_pca9685(self):
        self.bus.write_byte_data(self.i2c_address, 0x00, 0x10)
        time.sleep(0.005)
        prescale = int(25000000 / (4096 * 50) - 1)
        self.bus.write_byte_data(self.i2c_address, 0xFE, prescale)
        self.bus.write_byte_data(self.i2c_address, 0x00, 0x20)
        time.sleep(0.005)
        self.bus.write_byte_data(self.i2c_address, 0x01, 0x04)
        self._set_servo_pulse(0, self.center_pulse)
        self._set_servo_pulse(1, self.center_pulse)

    def _set_servo_pulse(self, channel, pulse_us):
        pulse_us = max(1000, min(2000, pulse_us))
        pwm_value = int((pulse_us * 4096) / 20000)
        base_reg = 0x06 + 4 * channel
        self.bus.write_byte_data(self.i2c_address, base_reg, 0)
        self.bus.write_byte_data(self.i2c_address, base_reg + 1, 0)
        self.bus.write_byte_data(self.i2c_address, base_reg + 2, pwm_value & 0xFF)
        self.bus.write_byte_data(self.i2c_address, base_reg + 3, (pwm_value >> 8) & 0xFF)

    @traitlets.observe('steering')
    def _on_steering(self, change):
        scaled = change['new'] * self.steering_gain + self.steering_offset
        pulse_us = self.center_pulse + (scaled * 500)
        self._set_servo_pulse(self.steering_channel, pulse_us)

    @traitlets.observe('throttle')
    def _on_throttle(self, change):
        scaled = change['new'] * self.throttle_gain
        if scaled > 0:
            pulse_us = self.center_pulse - (scaled * 200)
        elif scaled < 0:
            pulse_us = self.center_pulse + (abs(scaled) * 200)
        else:
            pulse_us = self.center_pulse
        self._set_servo_pulse(self.throttle_channel, pulse_us)

def initialize_camera():
    from jetcam.csi_camera import CSICamera
    camera = CSICamera(width=224, height=224, capture_fps=21)
    camera.running = True
    time.sleep(3)
    return camera

def get_camera_image(camera):
    img = camera.value
    if img is not None and img.shape[:2] != (224, 224):
        img = cv2.resize(img, (224, 224))
    return img

def load_trained_model():
    model = torchvision.models.resnet18(pretrained=False)
    model.fc = torch.nn.Linear(512, 2)
    model.load_state_dict(torch.load('road_following_model_cpu.pth', map_location='cpu', weights_only=True))
    model.eval()
    return model

class DrivingParams:
    THROTTLE = 0.15
    STEERING_GAIN = 0.75
    STEERING_BIAS = 0.0
    MAX_STEERING = 1.0
    MIN_STEERING = -1.0
    UPDATE_RATE = 0.05

def start_autonomous_driving():
    model = load_trained_model()
    camera = initialize_camera()
    car = NvidiaRacecar()
    car.throttle = DrivingParams.THROTTLE

    try:
        while True:
            image = get_camera_image(camera)
            if image is None:
                continue
            processed = preprocess(image)
            with torch.no_grad():
                output = model(processed)
                steering = float(output[0][0])
            steering = steering * DrivingParams.STEERING_GAIN + DrivingParams.STEERING_BIAS
            steering = max(DrivingParams.MIN_STEERING, min(DrivingParams.MAX_STEERING, steering))
            car.steering = steering
            time.sleep(DrivingParams.UPDATE_RATE)
    finally:
        car.throttle = 0.0
        car.steering = 0.0
        camera.running = False
